In [121]:
import numpy as np
from torch import nn
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from typing import Tuple
from torch.optim import Adam
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import torch.utils.data as data

In [132]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv")
timeseries = df[["Passengers"]].values.astype('float32')

# train-test split for time series
train_size = int(len(timeseries) * 0.67)
test_size = len(timeseries) - train_size
train, test = timeseries[:train_size], timeseries[train_size:]

def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)

lookback = 4
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)

class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

model = AirModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

n_epochs = 2000
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 100 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = np.sqrt(loss_fn(y_pred, y_train))
        y_pred = model(X_test)
        test_rmse = np.sqrt(loss_fn(y_pred, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

with torch.no_grad():
    # shift train predictions for plotting
    train_plot = np.ones_like(timeseries) * np.nan
    y_pred = model(X_train)
    y_pred = y_pred[:, -1, :]
    train_plot[lookback:train_size] = model(X_train)[:, -1, :]
    # shift test predictions for plotting
    test_plot = np.ones_like(timeseries) * np.nan
    test_plot[train_size+lookback:len(timeseries)] = model(X_test)[:, -1, :]
# plot
plt.plot(timeseries)
plt.plot(train_plot, c='r')
plt.plot(test_plot, c='g')
plt.show()

/home/thomas/.cache/pypoetry/virtualenvs/simplify-deployment-2FnGvFJr-py3.12/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([8, 3, 1])) that is different to the input size (torch.Size([8, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (8) must match the size of tensor b (3) at non-singleton dimension 1

In [129]:
X_batch

tensor([[[171.],
         [180.],
         [193.],
         [181.]],

        [[413.],
         [405.],
         [355.],
         [306.]],

        [[242.],
         [233.],
         [267.],
         [269.]],

        [[199.],
         [199.],
         [184.],
         [162.]]])

In [130]:
y_batch

tensor([[[180.],
         [193.],
         [181.],
         [183.]],

        [[405.],
         [355.],
         [306.],
         [271.]],

        [[233.],
         [267.],
         [269.],
         [270.]],

        [[199.],
         [184.],
         [162.],
         [146.]]])

In [128]:
z = AirModel()
z(X_train[0])

tensor([[-0.1241],
        [-0.1009],
        [-0.0767],
        [-0.0678]], grad_fn=<AddmmBackward0>)

In [131]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv")
timeseries = df[["Passengers"]].values.astype('float32')

# train-test split for time series
train_size = int(len(timeseries) * 0.67)
test_size = len(timeseries) - train_size
train, test = timeseries[:train_size], timeseries[train_size:]

def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+lookback]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)

lookback = 4
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)

class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x[:, -1, :])  # Use the last LSTM output
        return x

model = AirModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

n_epochs = 2000
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 100 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = np.sqrt(loss_fn(y_pred, y_train))
        y_pred = model(X_test)
        test_rmse = np.sqrt(loss_fn(y_pred, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

def predict_sequence(model, input_seq, prediction_len):
    """Generate predictions using the model for a given input sequence
    
    Args:
        model: Trained LSTM model
        input_seq: Initial sequence to start predictions
        prediction_len: Number of future steps to predict
    """
    model.eval()
    seq = input_seq.tolist()
    for _ in range(prediction_len):
        with torch.no_grad():
            input_tensor = torch.tensor(seq[-lookback:]).unsqueeze(0).unsqueeze(-1)
            next_pred = model(input_tensor)
            seq.append(next_pred.item())
    return seq[-prediction_len:]

with torch.no_grad():
    # shift train predictions for plotting
    train_plot = np.ones_like(timeseries) * np.nan
    for i in range(lookback, train_size):
        input_seq = train[i-lookback:i]
        train_plot[i] = predict_sequence(model, input_seq, 1)[-1]
    # shift test predictions for plotting
    test_plot = np.ones_like(timeseries) * np.nan
    for i in range(train_size, len(timeseries)-lookback):
        input_seq = test[i-train_size-lookback:i-train_size]
        test_plot[i+lookback] = predict_sequence(model, input_seq, 1)[-1]
# plot
plt.plot(timeseries)
plt.plot(train_plot, c='r')
plt.plot(test_plot, c='g')
plt.show()

Epoch 0: train RMSE 228.5246, test RMSE 427.6672
Epoch 100: train RMSE 178.4978, test RMSE 375.2819
Epoch 200: train RMSE 139.5635, test RMSE 332.7840
Epoch 300: train RMSE 108.2481, test RMSE 295.6481
Epoch 400: train RMSE 83.9018, test RMSE 263.0090
Epoch 500: train RMSE 62.2674, test RMSE 229.3492
Epoch 600: train RMSE 48.1118, test RMSE 202.2089
Epoch 700: train RMSE 38.3067, test RMSE 178.9674
Epoch 800: train RMSE 32.0773, test RMSE 159.4522
Epoch 900: train RMSE 28.1738, test RMSE 143.3520
Epoch 1000: train RMSE 25.2328, test RMSE 130.3687
Epoch 1100: train RMSE 24.3560, test RMSE 127.6413
Epoch 1200: train RMSE 22.2876, test RMSE 115.8857
Epoch 1300: train RMSE 24.0142, test RMSE 128.0206
Epoch 1400: train RMSE 21.7598, test RMSE 114.7559
Epoch 1500: train RMSE 20.7706, test RMSE 107.5122
Epoch 1600: train RMSE 20.5597, test RMSE 107.7081
Epoch 1700: train RMSE 20.1209, test RMSE 102.6821
Epoch 1800: train RMSE 19.0818, test RMSE 98.5118
Epoch 1900: train RMSE 18.8481, test RMS

ValueError: LSTM: Expected input to be 2D or 3D, got 4D instead